In [ ]:
!pip install pyspark
import pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.4 MB 30 kB/s 
     |████████████████████████████████| 198 kB 63.9 MB/s 
  Created wheel for pyspark: filename=pyspark-3.2.1-py2.py3-none-any.whl size=281853642 sha256=f65e193f2299886eb8d2e2b888537288fde074aab25514aef5d880909dcb0cf1
  Stored in directory: /root/.cache/pip/wheels/9f/f5/07/7cd8017084dce4e93e84e92efd1e1d5334db05f2e83bcef74f
Successfully built pyspark


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Binomial logistic regression

In [ ]:
from pyspark.sql import SparkSession

spark=SparkSession.builder.appName('Logistic Regression').getOrCreate()

In [ ]:
from pyspark.ml.classification import LogisticRegression


In [ ]:

# Load training data
df = spark.read.format("libsvm").load("/content/drive/MyDrive/0.MKCE/20.ML with Spark/sample_libsvm_data.txt")


In [ ]:

lr = LogisticRegression(maxIter=10, 
                        regParam=0.3, 
                        elasticNetParam=0.8)


In [ ]:

# Fit the model
lrModel = lr.fit(df)


In [ ]:

# Print the coefficients and intercept for logistic regression
print("Coefficients: " + str(lrModel.coefficients))
print("Intercept: " + str(lrModel.intercept))


Coefficients: (692,[272,300,323,350,351,378,379,405,406,407,428,433,434,435,455,456,461,462,483,484,489,490,496,511,512,517,539,540,568],[-7.52068987138421e-05,-8.115773146847101e-05,3.814692771846369e-05,0.0003776490540424337,0.00034051483661944103,0.0005514455157343105,0.0004085386116096913,0.000419746733274946,0.0008119171358670028,0.0005027708372668751,-2.3929260406601844e-05,0.000574504802090229,0.0009037546426803721,7.818229700244018e-05,-2.1787551952912764e-05,-3.4021658217896256e-05,0.0004966517360637634,0.0008190557828370367,-8.017982139522704e-05,-2.7431694037836214e-05,0.0004810832226238988,0.00048408017626778765,-8.926472920011488e-06,-0.00034148812330427335,-8.950592574121486e-05,0.00048645469116892167,-8.478698005186209e-05,-0.0004234783215831763,-7.29653577763134e-05])
Intercept: -0.5991460286401435


In [ ]:

# We can also use the multinomial family for binary classification
mlr = LogisticRegression(maxIter=10, regParam=0.3, elasticNetParam=0.8, family="multinomial")


In [ ]:

# Fit the model
mlrModel = mlr.fit(df)


In [ ]:

# Print the coefficients and intercepts for logistic regression with multinomial family
print("Multinomial coefficients: " + str(mlrModel.coefficientMatrix))
print("Multinomial intercepts: " + str(mlrModel.interceptVector))

In [ ]:
from pyspark.ml.classification import LogisticRegression

# Extract the summary from the returned LogisticRegressionModel instance trained
# in the earlier example
trainingSummary = lrModel.summary


In [ ]:

# Obtain the objective per iteration
objectiveHistory = trainingSummary.objectiveHistory
print("objectiveHistory:")
for objective in objectiveHistory:
    print(objective)


In [ ]:

# Obtain the receiver-operating characteristic as a dataframe and areaUnderROC.
trainingSummary.roc.show()
print("areaUnderROC: " + str(trainingSummary.areaUnderROC))


In [ ]:

# Set the model threshold to maximize F-Measure
fMeasure = trainingSummary.fMeasureByThreshold
maxFMeasure = fMeasure.groupBy().max('F-Measure').select('max(F-Measure)').head()
bestThreshold = fMeasure.where(fMeasure['F-Measure'] == maxFMeasure['max(F-Measure)']) \
    .select('threshold').head()['threshold']
lr.setThreshold(bestThreshold)

Multinomial Logistics Regression

In [ ]:
from pyspark.ml.classification import LogisticRegression

# Load training data
df = spark.read.format("libsvm").load("/content/drive/MyDrive/0.MKCE/20.ML with Spark/sample_multiclass_classification_data.txt")
############################################
# Load training data
#training = spark \
#    .read \
#    .format("libsvm") \
#    .load("data/mllib/sample_multiclass_classification_data.txt")
#################################

In [ ]:

lr = LogisticRegression(maxIter=10, regParam=0.3, elasticNetParam=0.8)


In [ ]:

# Fit the model
lrModel = lr.fit(df)


In [ ]:

# Print the coefficients and intercept for multinomial logistic regression
print("Coefficients: \n" + str(lrModel.coefficientMatrix))
print("Intercept: " + str(lrModel.interceptVector))


In [ ]:

trainingSummary = lrModel.summary


In [ ]:

# Obtain the objective per iteration
objectiveHistory = trainingSummary.objectiveHistory
print("objectiveHistory:")
for objective in objectiveHistory:
    print(objective)


In [ ]:

# for multiclass, we can inspect metrics on a per-label basis
print("False positive rate by label:")
for i, rate in enumerate(trainingSummary.falsePositiveRateByLabel):
    print("label %d: %s" % (i, rate))


In [ ]:

print("True positive rate by label:")
for i, rate in enumerate(trainingSummary.truePositiveRateByLabel):
    print("label %d: %s" % (i, rate))


In [ ]:

print("Precision by label:")
for i, prec in enumerate(trainingSummary.precisionByLabel):
    print("label %d: %s" % (i, prec))


In [ ]:

print("Recall by label:")
for i, rec in enumerate(trainingSummary.recallByLabel):
    print("label %d: %s" % (i, rec))


In [ ]:

print("F-measure by label:")
for i, f in enumerate(trainingSummary.fMeasureByLabel()):
    print("label %d: %s" % (i, f))


In [ ]:

accuracy = trainingSummary.accuracy
falsePositiveRate = trainingSummary.weightedFalsePositiveRate
truePositiveRate = trainingSummary.weightedTruePositiveRate
fMeasure = trainingSummary.weightedFMeasure()
precision = trainingSummary.weightedPrecision
recall = trainingSummary.weightedRecall

In [ ]:

print("Accuracy: %s\nFPR: %s\nTPR: %s\nF-measure: %s\nPrecision: %s\nRecall: %s"
      % (accuracy, falsePositiveRate, truePositiveRate, fMeasure, precision, recall))

Decision tree classifier

In [ ]:
from pyspark.ml import Pipeline
from pyspark.ml.classification import DecisionTreeClassifier
from pyspark.ml.feature import StringIndexer, VectorIndexer
from pyspark.ml.evaluation import MulticlassClassificationEvaluator


In [ ]:

# Load the data stored in LIBSVM format as a DataFrame.
df = spark.read.format("libsvm").load("/content/drive/MyDrive/0.MKCE/20.ML with Spark/sample_libsvm_data.txt")


In [ ]:

# Index labels, adding metadata to the label column.
# Fit on whole dataset to include all labels in index.
labelIndexer = StringIndexer(inputCol="label", 
                             outputCol="indexedLabel").fit(df)
# Automatically identify categorical features, and index them.
# We specify maxCategories so features with > 4 distinct values are treated as continuous.
featureIndexer =\
    VectorIndexer(inputCol="features", 
                  outputCol="indexedFeatures",
                  maxCategories=4).fit(df)


In [ ]:

# Split the data into training and test sets (30% held out for testing)
(trainingData, testData) = df.randomSplit([0.7, 0.3])


In [ ]:

# Train a DecisionTree model.
dt = DecisionTreeClassifier(labelCol="indexedLabel", 
                            featuresCol="indexedFeatures")


In [ ]:

# Chain indexers and tree in a Pipeline
pipeline = Pipeline(stages=[labelIndexer, 
                            featureIndexer, 
                            dt])


In [ ]:

# Train model.  This also runs the indexers.
model = pipeline.fit(trainingData)


In [ ]:

# Make predictions.
predictions = model.transform(testData)


In [ ]:

# Select example rows to display.
predictions.select("prediction", 
                   "indexedLabel", 
                   "features").show(5)


+----------+------------+--------------------+
|prediction|indexedLabel|            features|
+----------+------------+--------------------+
|       1.0|         1.0|(692,[98,99,100,1...|
|       1.0|         1.0|(692,[123,124,125...|
|       1.0|         1.0|(692,[124,125,126...|
|       1.0|         1.0|(692,[124,125,126...|
|       1.0|         1.0|(692,[124,125,126...|
+----------+------------+--------------------+
only showing top 5 rows



In [ ]:

# Select (prediction, true label) and compute test error
evaluator = MulticlassClassificationEvaluator(
    labelCol="indexedLabel", 
    predictionCol="prediction", 
    metricName="accuracy")

accuracy = evaluator.evaluate(predictions)


In [ ]:
accuracy

1.0

In [ ]:
print("Test Error = %g " % (1.0 - accuracy))


Test Error = 0 


In [ ]:

treeModel = model.stages[2]
# summary only
print(treeModel)

DecisionTreeClassificationModel: uid=DecisionTreeClassifier_f463ca5eff8e, depth=2, numNodes=5, numClasses=2, numFeatures=692


Random forest classifier

In [ ]:
from pyspark.ml import Pipeline
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.feature import IndexToString, StringIndexer, VectorIndexer
from pyspark.ml.evaluation import MulticlassClassificationEvaluator


In [ ]:

# Load and parse the data file, converting it to a DataFrame.
df = spark.read.format("libsvm").load("/content/drive/MyDrive/0.MKCE/20.ML with Spark/sample_libsvm_data.txt")


In [ ]:

# Index labels, adding metadata to the label column.
# Fit on whole dataset to include all labels in index.
labelIndexer = StringIndexer(inputCol="label", 
                             outputCol="indexedLabel").fit(df)


In [ ]:

# Automatically identify categorical features, and index them.
# Set maxCategories so features with > 4 distinct values are treated as continuous.
featureIndexer =\
    VectorIndexer(inputCol="features", 
                  outputCol="indexedFeatures", 
                  maxCategories=4).fit(df)


In [ ]:

# Split the data into training and test sets (30% held out for testing)
(trainingData, testData) = df.randomSplit([0.7, 0.3])


In [ ]:

# Train a RandomForest model.
rf = RandomForestClassifier(labelCol="indexedLabel", 
                            featuresCol="indexedFeatures",
                            numTrees=10)


In [ ]:

# Convert indexed labels back to original labels.
labelConverter = IndexToString(inputCol="prediction", 
                               outputCol="predictedLabel",
                               labels=labelIndexer.labels)


In [ ]:
# Chain indexers and forest in a Pipeline
pipeline = Pipeline(stages=[labelIndexer, 
                            featureIndexer, 
                            rf, 
                            labelConverter])


In [ ]:

# Train model.  This also runs the indexers.
model = pipeline.fit(trainingData)


In [ ]:

# Make predictions.
predictions = model.transform(testData)


In [ ]:

# Select example rows to display.
predictions.select("predictedLabel", 
                   "label", 
                   "features").show(5)


+--------------+-----+--------------------+
|predictedLabel|label|            features|
+--------------+-----+--------------------+
|           0.0|  0.0|(692,[95,96,97,12...|
|           0.0|  0.0|(692,[100,101,102...|
|           0.0|  0.0|(692,[122,123,148...|
|           0.0|  0.0|(692,[124,125,126...|
|           0.0|  0.0|(692,[128,129,130...|
+--------------+-----+--------------------+
only showing top 5 rows



In [ ]:
# Select (prediction, true label) and compute test error
evaluator = MulticlassClassificationEvaluator(
    labelCol="indexedLabel", 
    predictionCol="prediction", 
    metricName="accuracy")

print(accuracy)
accuracy = evaluator.evaluate(predictions)
print("Test Error = %g" % (1.0 - accuracy))


1.0
Test Error = 0


In [ ]:
rfModel = model.stages[2]
print(rfModel)  # summary only

RandomForestClassificationModel: uid=RandomForestClassifier_8be74521cd7b, numTrees=10, numClasses=2, numFeatures=692


Gradient-boosted tree classifier

In [ ]:
from pyspark.ml import Pipeline
from pyspark.ml.classification import GBTClassifier
from pyspark.ml.feature import StringIndexer, VectorIndexer
from pyspark.ml.evaluation import MulticlassClassificationEvaluator


In [ ]:

# Load and parse the data file, converting it to a DataFrame.
df = spark.read.format("libsvm").load("/content/drive/MyDrive/0.MKCE/20.ML with Spark/sample_libsvm_data.txt")


In [ ]:

# Index labels, adding metadata to the label column.
# Fit on whole dataset to include all labels in index.
labelIndexer = StringIndexer(inputCol="label", outputCol="indexedLabel").fit(df)
# Automatically identify categorical features, and index them.
# Set maxCategories so features with > 4 distinct values are treated as continuous.
featureIndexer =\
    VectorIndexer(inputCol="features", outputCol="indexedFeatures", maxCategories=4).fit(df)


In [ ]:

# Split the data into training and test sets (30% held out for testing)
(trainingData, testData) = df.randomSplit([0.7, 0.3])


In [ ]:

# Train a GBT model.
gbt = GBTClassifier(labelCol="indexedLabel", featuresCol="indexedFeatures", maxIter=10)


In [ ]:

# Chain indexers and GBT in a Pipeline
pipeline = Pipeline(stages=[labelIndexer, featureIndexer, gbt])


In [ ]:

# Train model.  This also runs the indexers.
model = pipeline.fit(trainingData)


In [ ]:

# Make predictions.
predictions = model.transform(testData)


In [ ]:

# Select example rows to display.
predictions.select("prediction", "indexedLabel", "features").show(5)


In [ ]:

# Select (prediction, true label) and compute test error
evaluator = MulticlassClassificationEvaluator(
    labelCol="indexedLabel", predictionCol="prediction", metricName="accuracy")
accuracy = evaluator.evaluate(predictions)
print("Test Error = %g" % (1.0 - accuracy))


In [ ]:

gbtModel = model.stages[2]
print(gbtModel)  # summary only

Multilayer perceptron classifier

In [ ]:
from pyspark.ml.classification import MultilayerPerceptronClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator


In [ ]:

# Load training data
df = spark.read.format("libsvm")\
    .load("/content/drive/MyDrive/0.MKCE/20.ML with Spark/sample_multiclass_classification_data.txt")


In [ ]:

# Split the data into train and test
splits = df.randomSplit([0.6, 0.4], 1234)
train = splits[0]
test = splits[1]


In [ ]:

# specify layers for the neural network:
# input layer of size 4 (features), two intermediate of size 5 and 4
# and output of size 3 (classes)
layers = [4, 5, 4, 3]


In [ ]:

# create the trainer and set its parameters
trainer = MultilayerPerceptronClassifier(maxIter=100, layers=layers, blockSize=128, seed=1234)


In [ ]:

# train the model
model = trainer.fit(train)


In [ ]:

# compute accuracy on the test set
result = model.transform(test)
predictionAndLabels = result.select("prediction", "label")
evaluator = MulticlassClassificationEvaluator(metricName="accuracy")


In [ ]:
print("Test set accuracy = " + str(evaluator.evaluate(predictionAndLabels)))

Linear Support Vector Machine Classifier

In [ ]:
from pyspark.ml.classification import LinearSVC


In [ ]:

# Load training data
training = spark.read.format("libsvm").load("/content/drive/MyDrive/0.MKCE/20.ML with Spark/sample_libsvm_data.txt")


In [ ]:

lsvc = LinearSVC(maxIter=10, regParam=0.1)


In [ ]:

# Fit the model
lsvcModel = lsvc.fit(training)


In [ ]:

# Print the coefficients and intercept for linear SVC
print("Coefficients: " + str(lsvcModel.coefficients))
print("Intercept: " + str(lsvcModel.intercept))

Naive Bayes

In [ ]:
from pyspark.ml.classification import NaiveBayes
from pyspark.ml.evaluation import MulticlassClassificationEvaluator


In [ ]:

# Load training data
data = spark.read.format("libsvm") \
    .load("/content/drive/MyDrive/0.MKCE/20.ML with Spark/sample_libsvm_data.txt")


In [ ]:

# Split the data into train and test
splits = data.randomSplit([0.6, 0.4], 1234)
train = splits[0]
test = splits[1]


In [ ]:

# create the trainer and set its parameters
nb = NaiveBayes(smoothing=1.0, modelType="multinomial")


In [ ]:

# train the model
model = nb.fit(train)


In [ ]:

# select example rows to display.
predictions = model.transform(test)
predictions.show()


In [ ]:

# compute accuracy on the test set
evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction",
                                              metricName="accuracy")


In [ ]:
accuracy = evaluator.evaluate(predictions)

In [ ]:

print("Test set accuracy = " + str(accuracy))